In [35]:
import os, sys, re, numpy as np, pandas as pd
from pyteomics import mzxml, mzml


class progressBar:
    def __init__(self, total):
        self.total = total
        self.barLength = 20
        self.count = 0
        self.progress = 0
        self.block = 0
        self.status = ""

    def increment(self):
        self.count += 1
        self.progress = self.count / self.total
        self.block = int(round(self.barLength * self.progress))
        if self.progress == 1:
            self.status = "Done...\r\n"
        else:
            self.status = ""
        #         self.status = str(self.count) + "/" + str(self.total)
        text = "\r  Progress: [{0}] {1}% {2}".format("#" * self.block + "-" * (self.barLength - self.block),
                                                     int(self.progress * 100), self.status)
        sys.stdout.write(text)
        sys.stdout.flush()


def getParams(paramFile):
    parameters = dict()
    with open(paramFile, 'r') as file:
        for line in file:
            if re.search(r'^#', line) or re.search(r'^\s', line):
                continue
            line = re.sub(r'#.*', '', line)  # Remove comments (start from '#')
            line = re.sub(r'\s*', '', line)  # Remove all whitespaces

            # Exception for "feature_files" parameter
            if "feature_files" in parameters and line.endswith("feature"):
                parameters["feature_files"].append(line)
            else:
                key = line.split('=')[0]
                val = line.split('=')[1]
                if key == "feature_files":
                    parameters[key] = [val]
                else:
                    parameters[key] = val
    return parameters


def parseIdtxt(idTxt, params):
    df = pd.read_table(idTxt, sep=";", skiprows=0, header=1)
    df = df.groupby(["Peptide", "Protein", "Outfile"]).size().reset_index()
#     outfiles = sorted(list(set(df["Outfile"]))) # Unique PSMs (i.e. MS2 scan numbers)
    psms = {}
    pep2psm = {}
    prot2psm = {}
    for i in range(df.shape[0]):
        pep = df["Peptide"][i]
        prot = df["Protein"][i]
        psm = df["Outfile"][i]
        fraction = re.sub(r"(.\d+)$", ".mzXML", os.path.dirname(psm))
        
        if fraction in psms:
            psms[fraction].append(psm)
        else:
            psms[fraction] = [psm]
        
        if fraction in pep2psm:
            if pep in pep2psm[fraction]:
                pep2psm[fraction][pep].append(psm)
                pep2psm[fraction][pep] = list(set(pep2psm[fraction][pep]))
            else:
                pep2psm[fraction][pep] = [psm]
        else:
            pep2psm[fraction] = {}
            pep2psm[fraction][pep] = [psm]

        if fraction in prot2psm:
            if prot in prot2psm[fraction]:
                prot2psm[fraction][prot].append(psm)
                prot2psm[fraction][prot] = list(set(prot2psm[fraction][prot]))
            else:
                prot2psm[fraction][prot] = [psm]
        else:
            prot2psm[fraction] = {}
            prot2psm[fraction][prot] = [psm]        

#         if pep in pep2psm:
#             pep2psm[pep].append(psm)
#             pep2psm[pep] = list(set(pep2psm[pep]))
#         else:
#             pep2psm[pep] = [psm]
#         if prot in prot2psm:
#             prot2psm[prot].append(psm)
#             prot2psm[prot] = list(set(prot2psm[prot]))
#         else:
#             prot2psm[prot] = [psm]

    return psms, pep2psm, prot2psm


def getReporterMz(name):
    if name == "sig126":
        return 126.127726
    elif name == "sig127" or name == "sig127N":
        return 127.124761
    elif name == "sig127C":
        return 127.131081
    elif name == "sig128N":
        return 128.128116
    elif name == "sig128" or name == "sig128C":
        return 128.134436
    elif name == "sig129" or name == "sig129N":
        return 129.131471
    elif name == "sig129C":
        return 129.137790
    elif name == "sig130N":
        return 130.134825
    elif name == "sig130" or name == "sig130C":
        return 130.141145
    elif name == "sig131" or name == "sig131N":
        return 131.138180
    elif name == "sig131C":
        return 131.144500
    elif name == "sig132N":
        return 132.141535
    elif name == "sig132C":
        return 132.147855
    elif name == "sig133N":
        return 133.144890
    elif name == "sig133C":
        return 133.151210
    elif name == "sig134N":
        return 134.148245


def getReporterIntensity(scanNum, params, **kwargs):
    tol = 10
    spec = reader[scanNum]
    reporterNames = params["tmt_reporters_used"].split(";")

    # Extract reporter ion m/z and intensity values
    mzArray = []
    intensityArray = []
    for reporter in reporterNames:
        if reporter in kwargs:
            mz = getReporterMz(reporter) * (1 + kwargs[reporter] / 1e6)
        else:
            mz = getReporterMz(reporter)
        lL = mz - mz * tol / 1e6
        uL = mz + mz * tol / 1e6
        ind = np.where((spec["m/z array"] >= lL) & (spec["m/z array"] <= uL))[0]
        if len(ind) == 0:
            reporterMz = 0
        elif len(ind) == 1:
            ind = ind[0]
            reporterMz = spec["m/z array"][ind]
        elif len(ind) > 1:
            ind2 = np.argmax(spec["intensity array"][ind])
            ind = ind[ind2]
            reporterMz = spec["m/z array"][ind]
        if lL <= reporterMz < uL:
            reporterIntensity = spec["intensity array"][ind]
        else:
            reporterIntensity = 0
        mzArray.append(reporterMz)
        intensityArray.append(reporterIntensity)

    # Extract K-TMT and R-y1 ion intensities
    tmtNo = re.search("\d+", os.path.basename(params["impurity_matrix"])).group(0)
    if tmtNo == "16":
        Kmz = 451.3199494907  # m/z of K-TMT-y1 ion for TMT16 (= 147.1128041645 + 304.2071453 (TMTpro-balancer))
    else:
        Kmz = 376.2757362992;  # m/z of K-TMT-y1 ion for TMT10, 11 (= 147.1128... + 229.1629321 (TMT-balancer))

    Kintensity = 0
    KlL = Kmz - Kmz * tol / 1e6
    KuL = Kmz + Kmz * tol / 1e6
    ind = np.where((spec["m/z array"] >= KlL) & (spec["m/z array"] <= KuL))[0]
    if len(ind) > 1:
        ind2 = np.argmax(spec["intensity array"][ind])
        ind = ind[ind2]
        ind = ind[0]
        Kintensity = spec["intensity array"][ind]

    Rintensity = 0
    Rmz = 175.1189521741
    RlL = Rmz - Rmz * tol / 1e6
    RuL = Rmz + Rmz * tol / 1e6
    ind = np.where((spec["m/z array"] >= RlL) & (spec["m/z array"] <= RuL))[0]
    if len(ind) > 1:
        ind2 = np.argmax(spec["intensity array"][ind])
        ind = ind[ind2]
        ind = ind[0]
        Rintensity = spec["intensity array"][ind]

    outArray = mzArray + intensityArray + [Kintensity, Rintensity]
    return outArray


def matchMs2ToMs3(psms, reader):
    output = {} # Dictionary whose key = MS2 scan number (string), value = corresponding MS3 scan number (string)
    for psm in psms:
        ms2Num = int(os.path.basename(psm).split(".")[-3])  # It should be changed to "int" here to deal with numbers such as "09999"
        ms2PrecMz = reader[str(ms2Num)]["precursorMz"][0]["precursorMz"]

        # Find corresponding MS3 scan number
        i = ms2Num + 1
        spec = reader[str(i)]
        while spec["msLevel"] > 1:
            if spec["msLevel"] == 3 and (ms2PrecMz - 0.1) < spec["precursorMz"][-1]["precursorMz"] < (ms2PrecMz + 3):
                ms3Num = spec["num"]
                output[str(ms2Num)] = ms3Num
                break
            i += 1
            spec = reader[str(i)]

    return output

In [67]:
import os, sys, re, numpy as np, pandas as pd
from pyteomics import mzxml, mzml


class progressBar:
    def __init__(self, total):
        self.total = total
        self.barLength = 20
        self.count = 0
        self.progress = 0
        self.block = 0
        self.status = ""

    def increment(self):
        self.count += 1
        self.progress = self.count / self.total
        self.block = int(round(self.barLength * self.progress))
        if self.progress == 1:
            self.status = "Done...\r\n"
        else:
            self.status = ""
        #         self.status = str(self.count) + "/" + str(self.total)
        text = "\r  Progress: [{0}] {1}% {2}".format("#" * self.block + "-" * (self.barLength - self.block),
                                                     int(self.progress * 100), self.status)
        sys.stdout.write(text)
        sys.stdout.flush()


def getParams(paramFile):
    parameters = dict()
    with open(paramFile, 'r') as file:
        for line in file:
            if re.search(r'^#', line) or re.search(r'^\s', line):
                continue
            line = re.sub(r'#.*', '', line)  # Remove comments (start from '#')
            line = re.sub(r'\s*', '', line)  # Remove all whitespaces

            # Exception for "feature_files" parameter
            if "feature_files" in parameters and line.endswith("feature"):
                parameters["feature_files"].append(line)
            else:
                key = line.split('=')[0]
                val = line.split('=')[1]
                if key == "feature_files":
                    parameters[key] = [val]
                else:
                    parameters[key] = val
    return parameters


def parseIdtxt(idTxt, params):
    df = pd.read_table(idTxt, sep=";", skiprows=0, header=1)
    df = df.groupby(["Peptide", "Protein", "Outfile"]).size().reset_index()
    psms = {}
    pep2psm = {}
    prot2psm = {}
    for i in range(df.shape[0]):
        pep = df["Peptide"][i]
        prot = df["Protein"][i]
        psm = df["Outfile"][i]
        fraction = re.sub(r"(.\d+)$", ".mzXML", os.path.dirname(psm))
        
        
        
        fraction = "NCI-11plex-1-F1-f10268.mzXML"
        
        
        if fraction in psms:
            psms[fraction].append(psm)
            psms[fraction] = list(set(psms[fraction]))
        else:
            psms[fraction] = [psm]
        
        if pep in pep2psm:
            pep2psm[pep].append(psm)
            pep2psm[pep] = list(set(pep2psm[pep]))
        else:
            pep2psm[pep] = [psm]
        
        if prot in prot2psm:
            prot2psm[prot].append(psm)
            prot2psm[prot] = list(set(prot2psm[prot]))
        else:
            prot2psm[prot] = [psm]
        
#         if fraction in pep2psm:
#             if pep in pep2psm[fraction]:
#                 pep2psm[fraction][pep].append(psm)
#                 pep2psm[fraction][pep] = list(set(pep2psm[fraction][pep]))
#             else:
#                 pep2psm[fraction][pep] = [psm]
#         else:
#             pep2psm[fraction] = {}
#             pep2psm[fraction][pep] = [psm]

#         if fraction in prot2psm:
#             if prot in prot2psm[fraction]:
#                 prot2psm[fraction][prot].append(psm)
#                 prot2psm[fraction][prot] = list(set(prot2psm[fraction][prot]))
#             else:
#                 prot2psm[fraction][prot] = [psm]
#         else:
#             prot2psm[fraction] = {}
#             prot2psm[fraction][prot] = [psm]        

    return psms, pep2psm, prot2psm


# def matchMs2ToMs3(psms, reader):
#     output = {} # Dictionary whose key = MS2 scan number (string), value = corresponding MS3 scan number (string)
#     for psm in psms:
#         ms2Num = int(os.path.basename(psm).split(".")[-3])  # It should be changed to "int" here to deal with numbers such as "09999"
#         ms2PrecMz = reader[str(ms2Num)]["precursorMz"][0]["precursorMz"]

#         # Find corresponding MS3 scan number
#         i = ms2Num + 1
#         spec = reader[str(i)]
#         while spec["msLevel"] > 1:
#             if spec["msLevel"] == 3 and (ms2PrecMz - 0.1) < spec["precursorMz"][-1]["precursorMz"] < (ms2PrecMz + 3):
#                 ms3Num = spec["num"]
#                 output[str(ms2Num)] = ms3Num
#                 break
#             i += 1
#             spec = reader[str(i)]

#     return output

def matchMs2ToMs3(psms):
    output = {} # Dictionary whose key = MS2 scan number (string), value = corresponding MS3 scan number (string)
    for fraction in psms.keys():
        output[fraction] = {}
        reader = mzxml.MzXML(fraction)    
        for psm in psms[fraction]:
            ms2Num = int(os.path.basename(psm).split(".")[-3])  # It should be changed to "int" here to deal with numbers such as "09999"
            ms2PrecMz = reader[str(ms2Num)]["precursorMz"][0]["precursorMz"]

            # Find corresponding MS3 scan number
            i = ms2Num + 1
            spec = reader[str(i)]
            while spec["msLevel"] > 1:
                if spec["msLevel"] == 3 and (ms2PrecMz - 0.1) < spec["precursorMz"][-1]["precursorMz"] < (ms2PrecMz + 3):
                    ms3Num = spec["num"]
                    output[fraction][ms2Num] = ms3Num
                    break
                i += 1
                spec = reader[str(i)]

    return output



def getReporterMz(name):
    if name == "sig126":
        return 126.127726
    elif name == "sig127" or name == "sig127N":
        return 127.124761
    elif name == "sig127C":
        return 127.131081
    elif name == "sig128N":
        return 128.128116
    elif name == "sig128" or name == "sig128C":
        return 128.134436
    elif name == "sig129" or name == "sig129N":
        return 129.131471
    elif name == "sig129C":
        return 129.137790
    elif name == "sig130N":
        return 130.134825
    elif name == "sig130" or name == "sig130C":
        return 130.141145
    elif name == "sig131" or name == "sig131N":
        return 131.138180
    elif name == "sig131C":
        return 131.144500
    elif name == "sig132N":
        return 132.141535
    elif name == "sig132C":
        return 132.147855
    elif name == "sig133N":
        return 133.144890
    elif name == "sig133C":
        return 133.151210
    elif name == "sig134N":
        return 134.148245


def getReporterIntensity(scanNum, params, **kwargs):
    tol = 10
    spec = reader[scanNum]
    reporterNames = params["tmt_reporters_used"].split(";")

    # Extract reporter ion m/z and intensity values
    mzArray = []
    intensityArray = []
    for reporter in reporterNames:
        if reporter in kwargs:
            mz = getReporterMz(reporter) * (1 + kwargs[reporter] / 1e6)
        else:
            mz = getReporterMz(reporter)
        lL = mz - mz * tol / 1e6
        uL = mz + mz * tol / 1e6
        ind = np.where((spec["m/z array"] >= lL) & (spec["m/z array"] <= uL))[0]
        if len(ind) == 0:
            reporterMz = 0
        elif len(ind) == 1:
            ind = ind[0]
            reporterMz = spec["m/z array"][ind]
        elif len(ind) > 1:
            ind2 = np.argmax(spec["intensity array"][ind])
            ind = ind[ind2]
            reporterMz = spec["m/z array"][ind]
        if lL <= reporterMz < uL:
            reporterIntensity = spec["intensity array"][ind]
        else:
            reporterIntensity = 0
        mzArray.append(reporterMz)
        intensityArray.append(reporterIntensity)

    # Extract K-TMT and R-y1 ion intensities
    tmtNo = re.search("\d+", os.path.basename(params["impurity_matrix"])).group(0)
    if tmtNo == "16":
        Kmz = 451.3199494907  # m/z of K-TMT-y1 ion for TMT16 (= 147.1128041645 + 304.2071453 (TMTpro-balancer))
    else:
        Kmz = 376.2757362992;  # m/z of K-TMT-y1 ion for TMT10, 11 (= 147.1128... + 229.1629321 (TMT-balancer))

    Kintensity = 0
    KlL = Kmz - Kmz * tol / 1e6
    KuL = Kmz + Kmz * tol / 1e6
    ind = np.where((spec["m/z array"] >= KlL) & (spec["m/z array"] <= KuL))[0]
    if len(ind) > 1:
        ind2 = np.argmax(spec["intensity array"][ind])
        ind = ind[ind2]
        ind = ind[0]
        Kintensity = spec["intensity array"][ind]

    Rintensity = 0
    Rmz = 175.1189521741
    RlL = Rmz - Rmz * tol / 1e6
    RuL = Rmz + Rmz * tol / 1e6
    ind = np.where((spec["m/z array"] >= RlL) & (spec["m/z array"] <= RuL))[0]
    if len(ind) > 1:
        ind2 = np.argmax(spec["intensity array"][ind])
        ind = ind[ind2]
        ind = ind[0]
        Rintensity = spec["intensity array"][ind]

    outArray = mzArray + intensityArray + [Kintensity, Rintensity]
    return outArray


In [68]:
# mzxmlFile = "NCI-11plex-1-F1-f10268.mzXML"
paramFile = "jump_qc.params"
# idTxt = "./sum_NCI-11Plex-1-F1-f10268/ID.txt"
idTxt = "ID.txt"


params = getParams(paramFile)
psms, pep2psm, prot2psm = parseIdtxt(idTxt, params)

In [69]:
ms2ToMs3 = matchMs2ToMs3(psms)

In [697]:
#####################################
# In case of MS3 quantification,    #
# MS2 - MS3 scans need to be paired #
#####################################
ms2ToMs3 = matchMs2ToMs3(psms)

####################################################
# Extract TMT reporter ion intensities - 1st round #
####################################################
qdict = {}  # Dictionary; key = MS2 scan number, value = TMT reporter intensities (array)
for fraction in sorted(ms2ToMs3.keys()):
    print("  TMT reporter ion extraction from %s" % os.path.basename(fraction))
    for ms2, ms3 in ms2ToMs3[fraction].items():
        reporterIntensity = getReporterIntensity(ms3, params)
        qdict[ms2] = reporterIntensity

# Create a dataFrame after the first extraction of reporter m/z and intensity values
reporters = params["tmt_reporters_used"].split(";")
columnNames = [re.sub("sig", "mz", i) for i in reporters] + reporters  + ["K_y1", "R_y1"]
df = pd.DataFrame.from_dict(qdict, orient = 'index', columns = columnNames) # Pandas dataframe of TMT reporter intensities

# Calculate m/z shift of each reporter ion (to be used for refining reporter intensities)
mzShifts = {}
nTot = df.shape[0]
for reporter in reporters:
    reporterMz = getReporterMz(reporter)
    measuredMz = df[re.sub("sig", "mz", reporter)][df[re.sub("sig", "mz", reporter)] > 0]   # Exclude un-quantified ones
    n = len(measuredMz)
    print("  %s\t%d (%.2f%%) matched" % (reporter, n, n / nTot * 100))
    mzShift = ((measuredMz - reporterMz) / reporterMz * 1e6).mean()
    mzShifts[reporter] = mzShift

##########################################################
# Refine the reporter intensities considering m/z shifts #
##########################################################
qdict = {}
for fraction in sorted(ms2ToMs3.keys()):
    for ms2, ms3 in ms2ToMs3[fraction].items():
        reporterIntensity = getReporterIntensity(ms3, params, **mzShifts)
        qdict[ms2] = reporterIntensity
df = pd.DataFrame.from_dict(qdict, orient = 'index', columns = columnNames)

# TMT impurity correction
dfImpurity = pd.read_table("TMT11.ini", sep="\t", skiprows=1, header=None, index_col = 0)
dfImpurity = pd.DataFrame(np.linalg.pinv(dfImpurity.values), dfImpurity.columns, dfImpurity.index)
dfImpurity.columns = [str("sig") + i for i in dfImpurity.columns]
df2 = df[reporters].dot(dfImpurity.T)
df2.columns = reporters
df2[df2 <= 1] = 0 # Lower bound to 0 (1 is a margin to prevent underflow when taking log2)
df[reporters] = pd.concat([df[reporters]/2, df2]).max(level = 0) # Upper bound to the half of original values (i.e. before correction)

  sig126	985 (98.50%) matched
  sig127N	975 (97.50%) matched
  sig127C	967 (96.70%) matched
  sig128N	985 (98.50%) matched
  sig128C	982 (98.20%) matched
  sig129N	981 (98.10%) matched
  sig129C	988 (98.80%) matched
  sig130N	973 (97.30%) matched
  sig130C	985 (98.50%) matched
  sig131N	960 (96.00%) matched
  sig131C	993 (99.30%) matched


In [ ]:
'''
# Generate raw_..._(non)zero.txt files
# For this implementation, 
# 1. No zero-intensity filtering (keep all zero-intensity PSMs)
# 2. No PPI filtering (PPI information is not available in Comet results)
filterMethods = params["min_intensity_method"].split(",")
filterCutoffs = params["min_intensity_value"].split(",")
for i in range(len(filterMethods)):
    if filterMethods[i] == "1":
        df = df[df[reporters].min(axis = 1) >= float(filterCutoffs[i])]
    elif filterMethods[i] == "2":
        df = df[df[reporters].max(axis = 1) >= float(filterCutoffs[i])]
    elif filterMethods[i] == "3":
        df = df[df[reporters].mean(axis = 1) >= float(filterCutoffs[i])]
    elif filterMethods[i] == "4":
        df = df[df[reporters].median(axis = 1) >= float(filterCutoffs[i])]

# For this implementation, min_intensity_..._psm_1_2_filtered is not considered
'''

In [698]:
# Loading-bias calculation
# "subDf" is used to calculate the loading-bias informatio and normalization factors
noiseLevel = 1000
snRatio = params["SNratio_for_correction"]

snRatio = 0

subDf = df[reporters][(df[reporters] > noiseLevel * snRatio).prod(axis = 1).astype(bool)] # Zero-intensity PSMs are excluded

# Loading-bias calculation using trimmed-mean
# - Divide each PSM by its row-wise mean and then take log2
# - In this way, non-variant PSMs will have values close to 0
subDf = np.log2(subDf.divide(subDf.mean(axis = 1), axis = 0))
pctTrimmed = float(params["percentage_trimmed"])
n = 0
for reporter in reporters:
    if n == 0:
        ind = ((subDf[reporter] > subDf[reporter].quantile(pctTrimmed / 200)) & (subDf[reporter] < subDf[reporter].quantile(1 - pctTrimmed / 200)))
    else:
        ind = ind & ((subDf[reporter] > subDf[reporter].quantile(pctTrimmed / 200)) & (subDf[reporter] < subDf[reporter].quantile(1 - pctTrimmed / 200)))
    n += 1

meanIntensity = 2 ** subDf.loc[ind].mean(axis = 0) * 100
# To express SD as percentage in raw-scale, take the mean deviation of 2^sd and 2^(-sd)
# For example, sdVal = 0.2 for a specific reporter which represent +/-0.2 in log2-scale
# 2^0.2 = 1.1487, 2^(-0.2) = 0.87
# positive variation = 2^0.2 - 1 = 0.1487 (assuming the center is ideally 1 (=2^0))
# negative variation = 1 - 2^(-0.2) = 0.13
# mean variation (in raw-scale) = (0.1487 + 0.13) / 2 = 0.1394
# Let the standard deviation of the reporter be 13.94% (= 0.1394 * 100)
sdVal = subDf.loc[ind].std(axis = 0)
sdIntensity = ((2 ** sdVal - 1) + (1 - 2 ** (-sdVal))) / 2 * 100
semIntensity = sdIntensity / np.sqrt(sum(ind))

print("  Reporter\tMean[%]\tSD[%]\tSEM[%]\t#PSMs")
for i in range(len(reporters)):
    print ("  %s\t%.2f\t%.2f\t%.2f\t%d" % (reporters[i], meanIntensity[i], sdIntensity[i], semIntensity[i], sum(ind)))


  Reporter	Mean[%]	SD[%]	SEM[%]	#PSMs
  sig126	124.40	39.64	3.01	173
  sig127N	104.01	25.92	1.97	173
  sig127C	123.99	21.20	1.61	173
  sig128N	145.34	23.39	1.78	173
  sig128C	65.85	19.85	1.51	173
  sig129N	62.71	25.22	1.92	173
  sig129C	112.67	19.60	1.49	173
  sig130N	72.80	24.50	1.86	173
  sig130C	88.70	26.61	2.02	173
  sig131N	44.70	48.86	3.71	173
  sig131C	119.64	12.18	0.93	173


In [688]:
# Normalization (i.e. loading-bias correction)

doNormalization = params["loading_bias_correction"]
normalizationMethod = params["loading_bias_correction_method"]
if doNormalization == "1":
    if normalizationMethod == "1": # Trimmed-mean
        normFactor = subDf.loc[ind].mean(axis = 0) - np.mean(subDf.loc[ind].mean())
    elif normalizationMethod == "2": # Trimmed-median
        normFactor = subDf.loc[ind].median(axis = 0) - np.mean(subDf.loc[ind].median())
    
    # Normalize the dataframe, df
    psmMean = df[reporters].mean(axis = 1)
    df[reporters] = np.log2(df[reporters].divide(psmMean, axis = 0).replace(0, np.nan))
    df[reporters] = df[reporters] - normFactor
    df[reporters] = 2 ** df[reporters]
    df[reporters] = df[reporters].multiply(psmMean, axis = 0).replace(np.nan, 0)
    
print(df)

            mz126      mz127N      mz127C      mz128N      mz128C      mz129N  \
1743   126.127235  127.124329  127.130867  128.127960  128.134018  129.131119   
1785   126.127487  127.124008  127.130920  128.127579  128.133911  129.131271   
1788   126.127571  127.124374  127.130913  128.127899    0.000000  129.131073   
2060     0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
2112   126.127518  127.124268  127.130753    0.000000  128.133911    0.000000   
...           ...         ...         ...         ...         ...         ...   
14921  126.127609  127.124680  127.130775  128.128006  128.134094  129.131531   
14937  126.127510  127.124596  127.130760  128.127914  128.133987  129.131485   
14941  126.127586  127.124825  127.130882  128.127975  128.133972  129.131393   
14952  126.127480  127.124565  127.130623  128.127991  128.134109  129.131332   
14973  126.127487  127.124657  127.130806  128.127914  128.134048  129.131302   

           mz129C      mz13

In [738]:
# Summarization of PSMs to peptides/proteins
pepDict = {}
for pep, psms in pep2psm.items():
    psms = [i.split(".")[-3] for i in psms]
    subDf = df[reporters][df.index.isin(psms)]    
    if subDf.empty:
        continue
    else:
        # 1. Take log2 again for the summarization
        # 2. Representative mean values of PSMs
        # 3. Mean-centering across reporters and then take mean over PSMs
        # 4. Add back the representative mean 
        subDf = np.log2(df[reporters][df.index.isin(psms)].replace(0, np.nan))
        psmMean = subDf.mean(axis = 1).sort_values(ascending = False)
        if len(psmMean) > 3:
            repMean = np.mean(psmMean[0:3])
        else:
            repMean = np.mean(psmMean)
        pepDict[pep] = (2 ** (subDf.sub(psmMean, axis = 0).mean(axis = 0) + repMean)).replace(np.nan, 0).tolist()

protDict = {}
for prot, psms in prot2psm.items():
    psms = [i.split(".")[-3] for i in psms]
    subDf = df[reporters][df.index.isin(psms)]    
    if subDf.empty:
        continue
    else:
        # 1. Take log2 again for the summarization
        # 2. Representative mean values of PSMs
        # 3. Mean-centering across reporters and then take mean over PSMs
        # 4. Add back the representative mean 
        subDf = np.log2(df[reporters][df.index.isin(psms)].replace(0, np.nan))
        psmMean = subDf.mean(axis = 1).sort_values(ascending = False)
        if len(psmMean) > 3:
            repMean = np.mean(psmMean[0:3])
        else:
            repMean = np.mean(psmMean)
        protDict[prot] = (2 ** (subDf.sub(psmMean, axis = 0).mean(axis = 0) + repMean)).replace(np.nan, 0).tolist()


In [735]:
psms = ["1743", "1788", "2060", "2112"]
psms = ["2060"]
subDf = np.log2(df[reporters][df.index.isin(psms)].replace(0, np.nan))
psmMean = subDf.mean(axis = 1).sort_values(ascending = False)
if len(psmMean) > 3:
    repMean = np.mean(psmMean[0:3])
else:
    repMean = np.mean(psmMean)

(2 ** (subDf.sub(psmMean, axis = 0).mean(axis = 0) + repMean)).replace(np.nan, 0).tolist()
# subDf

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 698.4754145768168,
 0.0,
 689.6569468194513,
 0.0,
 543.8897821056173]

In [742]:
protDict["sp|P14136|GFAP_HUMAN"]

[8672.600123957794,
 42598.32396432284,
 33058.185444339666,
 10582.131903809353,
 36174.17373936337,
 11333.91503107109,
 44119.07268432173,
 90619.14183245438,
 40836.19071467221,
 20224.52576923634,
 34820.11387530317]

In [712]:
df[reporters][df.index.isin(psms)]

,sig126,sig127N,sig127C,sig128N,sig128C,sig129N,sig129C,sig130N,sig130C,sig131N,sig131C
1743,2491.761347,2853.252386,3846.551398,5162.429670,1751.503829,1376.757657,3101.246413,1959.612601,2265.832931,1036.492093,3311.244677
1788,65951.405895,901.663526,2276.582764,1373.628768,8.668486,1309.174219,1699.675175,1360.927079,1346.907356,0.000000,5989.974316
2060,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,698.475415,0.000000,689.656947,0.000000,543.889782
2112,9497.996337,911.319322,830.762767,0.000000,949.661123,0.000000,1162.267878,1016.421021,0.000000,689.532597,2472.432513


In [72]:
psms = ['/research/rgs01/project_space/penggrp/BrainTumor_JP/penggrp/surendhar/MS3quan_JiHoon/NCI-11plex-1-F1-f10268/NCI-11plex-1-F1-f10268.1/NCI-11plex-1-F1-f10268.26125.26125.3.out']

In [73]:
psms

['/research/rgs01/project_space/penggrp/BrainTumor_JP/penggrp/surendhar/MS3quan_JiHoon/NCI-11plex-1-F1-f10268/NCI-11plex-1-F1-f10268.1/NCI-11plex-1-F1-f10268.26125.26125.3.out']

In [74]:
a = [i for os.path.basename(i) in psms]

SyntaxError: can't assign to function call (<ipython-input-74-5d83a064a0dc>, line 1)

In [77]:
a = [os.path.basename(i).split(".")[0] + ".mzXML" + "_" + os.path.basename(i).split(".")[-3] for i in psms]

In [92]:
df = pd.read_table("ID.txt", sep=";", skiprows=0, header=1)

In [102]:
with open('ID.txt') as f:
    jumpfPath = f.readline()

jumpfPath = re.sub("misc/idsum.db", "", jumpfPath.split("=")[1]).strip()
print(jumpfPath)

/research/rgs01/project_space/penggrp/BrainTumor_JP/penggrp/surendhar/MS3quan_JiHoon/sum_Batch1_sample1/


In [52]:
import pandas as pd
df = pd.read_table("id_uni_prot.txt", sep = "\t", skiprows = 0, header = 1, index_col = 1)
print(df)

                                Protein Group#  \
Protein Accession #                              
co|CON_A2A4G1|Con               SJPG000001.001   
co|CON_A2A5Y0|Con               SJPG000001.002   
co|CON_ENSP00000377550|Con      SJPG000001.004   
co|CON_P02533|Con               SJPG000001.007   
co|CON_P05784|Con               SJPG000001.009   
...                                        ...   
sp|P04350|TBB4A_HUMAN           SJPG119534.001   
sp|P68371|TBB4B_HUMAN           SJPG119535.001   
sp|P16989|YBOX3_HUMAN           SJPG119841.001   
sp|Q6ZNC4|ZN704_HUMAN           SJPG120261.001   
tr|A0A087WXU7|A0A087WXU7_HUMAN  SJPG120301.006   

                                                              Protein Description  \
Protein Accession #                                                                 
co|CON_A2A4G1|Con               TREMBL:A2A4G1|Contaminant Tax_Id=10090 Gene_Sy...   
co|CON_A2A5Y0|Con               TREMBL:A2A5Y0|Contaminant Tax_Id=10090 Gene_Sy...   
co|CON_EN

In [55]:
infoColumns = ["ProteinDescription", "GN", r"PSM#", r"TotalPeptide#", r"UniquePeptide#", r"%Coverage",
               "PeptideoftheHighestScore", r"Run#", r"Scan#", "m/z", "z", "ppm", "Xcorr", "dCn", r"Q-value(%)"]
df.columns = df.columns.str.replace(' ', '')
infoDf = df[infoColumns]


In [36]:
d = {'col1': [1, 2, 4], 'col2': [3, 4, 7]}
df1 = pd.DataFrame(data=d, index=["a", "c", "b"])
d = {'col1': ["d", "e", "f"], 'col2': ["q", "w", "e"]}
df2 = pd.DataFrame(data=d, index=["c", "a", "e"])

In [37]:
print(df1)
print(df2)

   col1  col2
a     1     3
c     2     4
b     4     7
  col1 col2
c    d    q
a    e    w
e    f    e


In [38]:
df1.index.isin(df2.index)

array([ True,  True, False])

In [47]:
df3 = pd.concat([df1, df2], axis=1, join = "inner")

In [48]:
df3

,col1,col2,col1,col2
a,1,3,e,w
c,2,4,d,q


In [43]:
df4

,col1_x,col2_x,col1_y,col2_y
a,1.0,3.0,e,w
b,4.0,7.0,NaN,NaN
c,2.0,4.0,d,q
e,NaN,NaN,f,e
